# GEH Pharmacy Department

## Step 1: Import necessary packages

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import *

## Data Cleaning (April & May Data)

### Step 2: Read April & May data for merging

In [ ]:
files_xls_1 = ['../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (April 2021).xlsx', '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 May.xlsx']

In [ ]:
df = pd.DataFrame()

In [ ]:
for f in files_xls_1:
    data = pd.read_excel(f, 'Template')
    df = df.append(data)

In [ ]:
df

#### Step 3: Make first row as the column name

In [ ]:
df.columns = df.iloc[0]

In [ ]:
df = df.drop(0)
df

In [ ]:
df = df.dropna(axis=1, how='all')
df

#### Step 4: Drop row if case number is empty

In [ ]:
df = df[pd.notnull(df['Case No'])]

In [ ]:
df

In [ ]:
df.info()

#### Step 5: Drop empty column

In [ ]:
df2= df.copy()

In [ ]:
df2 = df2.drop(df.iloc[:, 17:], axis=1)

In [ ]:
#reset index 
df2 = df2.reset_index(drop=True)

#### Step 6: Rename common columns

In [ ]:
df2.rename({'Date ': 'Date', 
            'Case No': 'Case Number', 
            'Room No': 'Room/Bed',
            'SCM / Handwritten TTO ': 'TTO Type', 
            'No. of items':'No. of Drugs', 
            'Time in / Verified': 'DateTime TTO Received', # the start time
            'Time checked':'DateTime TTO Checked', 
            'Time reached ': 'DateTime TTO Dispensed', # the end time 
            ' Time Counselled':'DateTime (Counselled)', # end time if patients need to be counselled
           'Limo Service ':'Limo Service'},  
           axis=1, inplace=True)

In [ ]:
df2.info()

#### Step 7: Handle missing value

In [ ]:
list(df2.columns)

In [ ]:
df2.isnull().sum()

##### Missing date

In [ ]:
# change to datetime formate
df2['Date'] = pd.to_datetime(df2['Date'])

In [ ]:
missing_date = df2[df2['Date'].isnull()]
missing_date

In [ ]:
# fill up missing date based on previous known value 
df2['Date'] = df2['Date'].fillna(method='ffill')
df2.tail()

##### Missing Time TTO Received

In [ ]:
# drop the one missing row in april dataset bc no time in cant calculate time taken
# drop row in may too for same reason
df2 = df2.dropna(subset=['DateTime TTO Received'])

In [ ]:
df2.isnull().sum()

## EDA (April & May Data)

In [ ]:
df2

### Step 8: Get Day of Week

In [ ]:
df2['Weekday'] = df2['Date'].dt.day_name()

In [ ]:
# handle Unknown string format: 9.:09 in Time TTO reached
df2['DateTime TTO Dispensed'] = df2['DateTime TTO Dispensed'].replace('9.:09', 'NA')

In [ ]:
#tto_received = pd.to_datetime(df2['Time TTO Received'].astype(str)) 
#tto_reached = pd.to_datetime(df2[df2['Time TTO Reached'] != 'NA']['Time TTO Reached'].astype(str))
#tto_counselled = pd.to_datetime(df2[df2['Time TTO Counselled'] != 'NA']['Time TTO Counselled'].astype(str))

#df2['Overall Time Taken (mins)'] = tto_reached.sub(tto_received).dt.total_seconds().div(60)

In [ ]:
#df2.info()

In [ ]:
#df2['time_diff'] = pd.Timestamp(df2['Time TTO Received']) - pd.Timestamp(df['end_time'])   
#def f(row):
#    if row['Time TTO Counselled'] != " ":
#        val = tto_counselled.sub(tto_received).dt.total_seconds().div(60)
    #elif row['Time TTO Counselled'] <  row['Time TTO Reached']:
    #    val = tto_reached.sub(tto_received).dt.total_seconds().div(60)
#    else:
#        val = tto_reached.sub(tto_received).dt.total_seconds().div(60)
#    return val

In [ ]:
#df2['Overall Time Taken (mins)_2'] = df2.apply(f, axis=1)
#df2

## Step 9: Export to CSV for Final Merge (April & May)

In [ ]:
df2.to_csv("../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (April-May 2021).csv", index = False, header=True)

## Data Cleaning (Remaining Months Data)

### Step 10: Read remaining months for merging

In [ ]:
files_xls_2 = ['../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 June.xlsx',
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 July.xlsx',
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Aug.xlsx',
'../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Sep.xlsx',
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Nov.xlsx', 
'../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Oct.xlsx', 
 '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 Dec.xlsx']

In [ ]:
df3 = pd.DataFrame()
for f in files_xls_2:
    data = pd.read_excel(f)
    df3 = df3.append(data)
df3

In [ ]:
df3.info()

In [ ]:
#reset index 
df3 = df3.reset_index(drop=True)

#### Drop irrelevant columns

In [ ]:
# drop columns that should be ignored
df3 = df3.drop(['TTO_TAT_Calc', 'Total number of TTO', 'Status_Time_Met', 'ID', 'Title', 'TTOaddon', 'Item Type', 'Path', 'Unnamed: 23', 'Unnamed: 24'], axis = 1)
# drop other redundant columns
#df3 = df3.drop(['Pharmacist_Checked', 'Pharmacist_Dispensed', 'Dispatcher' ], axis = 1)

### Step 11: Rename common columns

In [ ]:
df3.info()

In [ ]:
df3.rename({'No. of Items':'No. of Drugs', 
            'Time In': 'DateTime TTO Received', # the start time
            'Time Checked':'DateTime TTO Checked', 
            'Time Reached': 'DateTime TTO Dispensed'}, # the end time  'Time Dispensed' : 'DateTime TTO Dispensed'},
           axis=1, inplace=True)

### Step 12: Remove date in time metrics

In [ ]:
columns = ['DateTime TTO Received', 'DateTime TTO Checked', 'DateTime TTO Dispensed']
           #, 'DateTime TTO Counselled']

for column in columns:
    # remove date in the time metric
    #df3[column]=df3[column].dt.time 
    df3[column] = df3[column].astype(str).str.split(' ').str[-1]

In [ ]:
df3

In [ ]:
df3.info()

### Step 13: Handle missing value

In [ ]:
df3.isnull().sum()

#### Missing Date

In [ ]:
# change to datetime format
df3['Date'] = pd.to_datetime(df3['Date'])

In [ ]:
missing_date = df3[df3['Date'].isnull()]
missing_date

In [ ]:
#drop rows
df3 = df3.dropna(subset=['Date'])

#### Missing Pharmacist Dispensed

In [ ]:
# missing pharmacist dispensed
missing_date = df3[df3['Pharmacist_Dispensed'].isnull()]
missing_date.head()

#### Missing No. of Drugs

In [ ]:
df3['No. of Drugs'].replace(np.nan, 'NA', inplace = True)

In [ ]:
missing_date = df3[df3['Time'].notnull()]
missing_date

In [ ]:
df3.isnull().sum()

## EDA (Remaining Months Data)

### Step 14: Get Day of Week

In [ ]:
# add weekday column
df3['Weekday'] = df3['Date'].dt.day_name()

## Step 15: Export to CSV for Final Merge

In [ ]:
df3.to_csv("../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (June-Dec 2021).csv", index = None, header=True)

## Data Cleaning (Final)

### Step 16: Final Merge of the above CSVs

In [ ]:
merged_df = pd.concat([df2, df3])

In [ ]:
merged_df

### Step 17: Clean datetime columns

In [ ]:
columns = ['DateTime TTO Received',
           'DateTime TTO Checked', 
           'DateTime TTO Dispensed']
for column in columns:
    merged_df.loc[:,column] = pd.to_datetime((merged_df['Date'].astype(str) + ' ' + merged_df[column].astype(str)), infer_datetime_format=True, errors='coerce')

In [ ]:
merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.normalize()

In [ ]:
merged_df.head()

In [ ]:
list(merged_df.columns)

### Step 18: Drop ununsed columns & rename columns

In [ ]:
merged_df = merged_df.drop(['Pharmacist_Checked','Pharmacist_Dispensed','Dispatcher',
                            'status_CheckedTTO','status_TTOReached','status_Dispensed','status_TTOIn','Time', 'Time Dispensed'],
                           axis=1)

In [ ]:
merged_df.rename({'Timings (Checked)' : 'Time Taken (Received to Checked)',
              'Timings (Reached)': 'Time Taken (Received to Dispensed)'},  
           axis=1, inplace=True)
    

In [ ]:
merged_df.info()

In [ ]:
merged_df[merged_df.index.duplicated()]

In [ ]:
#reset index 
merged_df = merged_df.reset_index(drop=True)

In [ ]:
merged_df

## EDA (Final)

### Step 19: Get Overall Time Taken (mins)

In [ ]:
#merged_df["DateTime TTO Dispensed"] = pd.to_datetime(merged_df[merged_df['DateTime TTO Dispensed'] != 'NA']['DateTime TTO Dispensed'].astype(str))
#merged_df["DateTime TTO Counselled"]= pd.to_datetime(merged_df[merged_df['DateTime TTO Counselled'] != 'NA']['DateTime TTO Counselled'].astype(str))

In [ ]:
#merged_df["TTO Last Timestamp"] = merged_df[['DateTime TTO Reached', 'DateTime TTO Counselled']].max(axis=1)
#merged_df["Last Timestamp"]  = merged_df['Last Timestamp'].dt.time

In [ ]:
#columns = ['Time TTO Reached', 'Time TTO Counselled', 'Last Timestamp']

#for column in column:
#    merged_df[column]  = merged_df[column].dt.time

In [ ]:
#merged_df["Time TTO Received"]  = pd.to_datetime(merged_df["Time TTO Received"] , infer_datetime_format=True, errors='coerce')
#merged_df["Time TTO Reached"]  = pd.to_datetime(merged_df["Time TTO Reached"] , infer_datetime_format=True, errors='coerce')
#merged_df["Time TTO Counselled"]  = pd.to_datetime(merged_df["Time TTO Counselled"] , infer_datetime_format=True, errors='coerce')

In [ ]:
#merged_df["DateTime TTO Dispensed"]  = merged_df['DateTime TTO Dispensed'].dt.time
#merged_df["DateTime TTO Counselled"]  = merged_df['DateTime TTO Counselled'].dt.time

In [ ]:
#merged_df["TTO Last Timestamp"]  = merged_df['TTO Last Timestamp'].dt.time

In [ ]:
#merged_df.info()

In [ ]:
#merged_df

In [ ]:
tto_received = pd.to_datetime(merged_df['DateTime TTO Received'].astype(str)) 
tto_dispensed = pd.to_datetime(merged_df[merged_df['DateTime TTO Dispensed'] != 'NA']['DateTime TTO Dispensed'].astype(str))
#tto_counselled = pd.to_datetime(df2[df2['Time TTO Counselled'] != 'NA']['Time TTO Counselled'].astype(str))

merged_df['TAT'] = tto_dispensed.sub(tto_received).dt.total_seconds().div(60)

In [ ]:
merged_df

### Step 19: Create Meet KPI column with yes/no value

In [ ]:
def f(row):
    if row['TAT'] > 45:
        val = 'No'
    elif row['TAT'] > 0 and row['TAT'] <= 45:
        val = 'Yes'
    else:
        val = 'NA'
    return val

In [ ]:
merged_df['Meet KPI'] = merged_df.apply(f, axis=1)
merged_df

In [ ]:
merged_df.info()

### Step 20: Create Office Hours column to check if case handled during offcie hours

In [ ]:
x = datetime.strptime('08:30:00', '%H:%M:%S').time()
x

In [ ]:
#merged_df["DateTime TTO Received"]  = pd.to_datetime(merged_df["DateTime TTO Received"].astype(str))
#merged_df["DateTime TTO Dispensed"]  = pd.to_datetime(merged_df["DateTime TTO Dispensed"].astype(str))
#merged_df["TTO Last Timestamp"]  = pd.to_datetime(merged_df["TTO Last Timestamp"] , infer_datetime_format=True, errors='coerce')

In [ ]:
#merged_df

In [ ]:
#merged_df["DateTime TTO Received"]   = merged_df["DateTime TTO Received"].dt.time
#merged_df["DateTime TTO Dispensed"]   = merged_df["DateTime TTO Dispensed"].dt.time

In [ ]:
merged_df['Office Hours'] = np.where(merged_df['DateTime TTO Received'].isnull() | merged_df['DateTime TTO Dispensed'].isnull() , 'NA',
                            np.where(  #(dfinal['Time TTO Received'] >= pd.to_datetime('08:30:00').time()) & 
                                        (merged_df['DateTime TTO Dispensed'].dt.time <= pd.to_datetime('20:00:00').time()),
                                        'Yes', 'No'))

merged_df

### Step 21: Create Average Time Taken from (Received to Checked) & (Checked to Dispensed) columns

In [ ]:
merged_df['Time Taken (Checked to Dispensed)'] = (merged_df['DateTime TTO Dispensed'] - merged_df['DateTime TTO Checked'])

#### Average time taken per day

In [ ]:
#Get Duration Difference of Received to Checked
merged_df['Time Taken (Received to Checked)'] = (merged_df['DateTime TTO Checked'] - merged_df['DateTime TTO Received'])
#Get Duration Difference of Checked to Dispensed
merged_df['Time Taken (Checked to Dispensed)'] = (merged_df['DateTime TTO Dispensed'] - merged_df['DateTime TTO Checked'])

In [ ]:
#Create new dataframe to get results of Average Time Taken for each (Received to Checked) & (Checked to Dispensed)
dfinal_avg_time = merged_df[['Date', 'Time Taken (Received to Checked)', 'Time Taken (Checked to Dispensed)']]
dfinal_avg_time.head()

In [ ]:
#Convert NaT to 00:00:00
dfinal_avg_time.loc[dfinal_avg_time['Time Taken (Received to Checked)'].isnull(), 'Time Taken (Received to Checked)'] = pd.to_timedelta(0)
dfinal_avg_time.loc[dfinal_avg_time['Time Taken (Checked to Dispensed)'].isnull(), 'Time Taken (Checked to Dispensed)'] = pd.to_timedelta(0)

In [ ]:
#Re-instate time format
#dfinal_avg_time['Time Taken (Received to Checked)'] = pd.to_datetime(dfinal_avg_time['Time Taken (Received to Checked)'], infer_datetime_format=True).dt.time
#dfinal_avg_time['Time Taken (Checked to Dispensed)'] = pd.to_datetime(dfinal_avg_time['Time Taken (Checked to Dispensed)'], infer_datetime_format=True).dt.time

#Get average time taken per day
dfinal_avg_time = dfinal_avg_time.groupby(pd.to_datetime(dfinal_avg_time['Date']).dt.date).mean(numeric_only=False)
dfinal_avg_time

In [ ]:
#Remove miliseconds
dfinal_avg_time['Time Taken (Received to Checked)'] = dfinal_avg_time['Time Taken (Received to Checked)'].dt.floor('s')
dfinal_avg_time['Time Taken (Checked to Dispensed)'] = dfinal_avg_time['Time Taken (Checked to Dispensed)'].dt.floor('s')

dfinal_avg_time

In [ ]:
#Rename Columns
dfinal_avg_time = dfinal_avg_time.rename({'Time Taken (Received to Checked)' : 'Avg Time Taken / Day (Received to Checked)', 'Time Taken (Checked to Dispensed)' : 'Avg Time Taken / Day (Checked to Dispensed)'}, axis=1)

In [ ]:
dfinal_avg_time = dfinal_avg_time.reset_index(drop=True)
dfinal_avg_time

In [ ]:
#Merge the results from grouby for Avg Time Taken
merged_df = merged_df.merge(dfinal_avg_time, on='Date', how='left')
merged_df

In [ ]:
# drop Time Taken (Received to Dispensed) since it is the same as TAT
merged_df.drop(['Time Taken (Received to Dispensed)'], axis=1, inplace=True)

In [ ]:
# replace HW TTO with handwritten 
merged_df['TTO Type'] = merged_df['TTO Type'].replace('HW TTO', 'Handwritten')

#### Average time taken per month 

In [ ]:
merged_df['Month']= pd.to_datetime(merged_df['Date']).dt.month_name()

In [ ]:
dfinal_avg_time = merged_df[['Date', 'Time Taken (Received to Checked)', 'Time Taken (Checked to Dispensed)']]

dfinal_avg_time['Month']= pd.to_datetime(dfinal_avg_time['Date']).dt.month_name()

#Get average time taken per month
dfinal_avg_time= dfinal_avg_time.groupby('Month').agg({'Time Taken (Received to Checked)': np.sum, 'Time Taken (Checked to Dispensed)': np.sum})

x = dfinal_avg_time['Time Taken (Received to Checked)'] / np.timedelta64(1, 'm')
y = merged_df.groupby('Month')['Time Taken (Received to Checked)'].count()

dfinal_avg_time['Avg Time Taken / Month (Received to Checked)'] = (x/y)

dfinal_avg_time['Avg Time Taken / Month (Received to Checked)'] = pd.to_datetime(dfinal_avg_time['Avg Time Taken / Month (Received to Checked)'], unit='m').apply(lambda x: x.strftime("%H:%M:%S"))

In [ ]:
a = dfinal_avg_time['Time Taken (Checked to Dispensed)'] / np.timedelta64(1, 'm')
b = merged_df.groupby('Month')['Time Taken (Checked to Dispensed)'].count()

dfinal_avg_time['Avg Time Taken / Month (Checked to Dispensed)'] = (a/b)

dfinal_avg_time['Avg Time Taken / Month (Checked to Dispensed)'] = pd.to_datetime(dfinal_avg_time['Avg Time Taken / Month (Checked to Dispensed)'], unit='m').apply(lambda x: x.strftime("%H:%M:%S"))

In [ ]:
dfinal_avg_time

In [ ]:
dfinal_avg_time = dfinal_avg_time.drop(dfinal_avg_time.columns[[0, 1]], axis=1)

In [ ]:
merged_df = merged_df.merge(dfinal_avg_time, on='Month', how='left')
merged_df

In [ ]:
merged_df['Avg Time Taken / Month (Received to Checked)'] = pd.to_timedelta(merged_df['Avg Time Taken / Month (Received to Checked)'])
merged_df['Avg Time Taken / Month (Checked to Dispensed)'] = pd.to_timedelta(merged_df['Avg Time Taken / Month (Checked to Dispensed)'])

In [ ]:
merged_df

### Step 22: Map Room/Bed No. to Ward No.

Match Room No. to the corresponding Ward No.

In [ ]:
df_ward = pd.read_excel("../../Data/Pharmacy Dept/GEH TTO Data/GEH Wards.xlsx", sheet_name=0)

df_ward

In [ ]:
df_ward.info()

In [ ]:
df_ward['Room'] = df_ward['Room'].astype(str)
merged_df['Room/Bed'] = merged_df['Room/Bed'].astype(str)

In [ ]:
dfinal = pd.merge(merged_df, df_ward, how='left', left_on=['Room/Bed'], right_on=['Room'])

dfinal.head()

In [ ]:
dfinal[dfinal['Ward_y'].isnull()]

#### Step 23: Fix Invalid Room

In [ ]:
#fixing values in Room that are actually referring to the Ward, and match its corresponding in the Ward column
dfinal.loc[(dfinal['Room/Bed'].str.contains('endo', case=False, na=False)), 'Ward_y'] = 'ENDO'
dfinal.loc[(dfinal['Room/Bed'].str.contains('6W', case=False, na=False)), 'Ward_y'] = '6W'
#dfinal.loc[(dfinal['Room'].str.contains('icu', case=False, na=False)), 'Ward_y'] = 'ICU'

#Fix values in Room/Bed that are string 'nan' to np.NaN
dfinal.loc[(dfinal['Room/Bed'].str.contains('nan', case=False, na=False)), 'Room/Bed'] = np.NaN

In [ ]:
dfinal.drop(['Ward_x', 'Room_y'], axis=1, inplace=True)
dfinal.rename({'Ward_y': 'Ward'}, axis=1, inplace=True)

In [ ]:
dfinal[dfinal['Ward'].isnull() & dfinal['Room/Bed'].notnull()]

In [ ]:
dfinal

## Step 24: Export Final CLEANED to csv 

In [ ]:
dfinal.to_csv("../../Data/Pharmacy Dept/Data Cleaning/GEH_Data_Cleaned.csv", index = None, header=True)